In [ ]:
import hashlib
import random
from math import gcd

def pow_mod(a, e, n):
    """Tính a^e mod n bằng bình phương và nhân (fast exponentiation)."""
    res = 1
    a %= n
    while e > 0:
        if e & 1:
            res = (res * a) % n
        a = (a * a) % n
        e >>= 1
    return res

def inv_mod(a, n):
    """Nghịch đảo modulo n (dùng extended Euclid). Trả về x sao cho a*x ≡ 1 (mod n)."""
    # extended gcd
    r0, r1 = a, n
    s0, s1 = 1, 0
    t0, t1 = 0, 1
    while r1 != 0:
        q = r0 // r1
        r0, r1 = r1, r0 - q * r1
        s0, s1 = s1, s0 - q * s1
        t0, t1 = t1, t0 - q * t1
    if r0 != 1:
        raise ValueError("Không tồn tại nghịch đảo modulo")
    return s0 % n

def sha1_int(msg: bytes) -> int:
    """Hash SHA-1 rồi chuyển thành số nguyên."""
    h = hashlib.sha1(msg).digest()
    return int.from_bytes(h, "big")

def is_probable_prime(n, k=8):
    """Miller-Rabin test đơn giản, đủ để demo."""
    if n < 2:
        return False
    # loại nhanh
    small_primes = [2, 3, 5, 7, 11, 13, 17, 19, 23]
    if n in small_primes:
        return True
    if any(n % p == 0 for p in small_primes):
        return False

    # viết n-1 = d * 2^s
    d = n - 1
    s = 0
    while d % 2 == 0:
        d //= 2
        s += 1

    for _ in range(k):
        a = random.randrange(2, n - 2)
        x = pow_mod(a, d, n)
        if x == 1 or x == n - 1:
            continue
        for _ in range(s - 1):
            x = pow_mod(x, 2, n)
            if x == n - 1:
                break
        else:
            return False
    return True

def gen_prime(bits):
    """Sinh một số nguyên tố có 'bits' bit (demo, không tối ưu)."""
    while True:
        # sinh số lẻ ngẫu nhiên với bit cao = 1
        n = random.getrandbits(bits) | 1 | (1 << (bits - 1))
        if is_probable_prime(n):
            return n


def gen_dsa_params(q_bits=160, p_bits=512):
    """
    Sinh tham số DSA kiểu classic:
    - q: nguyên tố ~160 bit
    - p: nguyên tố ~p_bits, sao cho p-1 chia hết cho q
    - g: phần tử sinh của nhóm con bậc q.
    """
    q = gen_prime(q_bits)

    # tìm p sao cho p = k*q + 1 là nguyên tố, gần p_bits
    while True:
        # chọn k sao cho p có ~p_bits bit
        k_min = 1 << (p_bits - q_bits - 1)
        k_max = 1 << (p_bits - q_bits)
        k = random.randrange(k_min, k_max)
        p = k * q + 1
        if is_probable_prime(p):
            break

    # tìm g
    while True:
        h = random.randrange(2, p - 1)
        g = pow_mod(h, (p - 1) // q, p)
        if g > 1:
            break

    return p, q, g

def dsa_keygen(p, q, g):
    """
    Sinh khóa DSA:
    - private x: 0 < x < q
    - public  y: g^x mod p
    """
    x = random.randrange(1, q)
    y = pow_mod(g, x, p)
    return x, y

def dsa_sign(message: bytes, p, q, g, x):
    """
    Ký DSA:
    - message: bytes
    - (p, q, g): tham số hệ
    - x: khóa bí mật
    Trả về (r, s)
    """
    H = sha1_int(message)
    while True:
        k = random.randrange(1, q)  # nonce bí mật, phải random và không được reuse
        r = pow_mod(g, k, p) % q
        if r == 0:
            continue
        try:
            k_inv = inv_mod(k, q)
        except ValueError:
            continue
        s = (k_inv * (H + x * r)) % q
        if s == 0:
            continue
        return r, s


def dsa_verify(message: bytes, signature, p, q, g, y) -> bool:
    """
    Kiểm tra chữ ký DSA.
    - message: bytes
    - signature: (r, s)
    - (p, q, g): tham số hệ
    - y: khóa công khai
    """
    r, s = signature
    if not (0 < r < q and 0 < s < q):
        return False
    H = sha1_int(message)
    try:
        w = inv_mod(s, q)
    except ValueError:
        return False

    u1 = (H * w) % q
    u2 = (r * w) % q

    v = (pow_mod(g, u1, p) * pow_mod(y, u2, p)) % p
    v = v % q
    return v == r

if __name__ == "__main__":
    # CẢNH BÁO: RẤT CHẬM vì p_bits=512, có thể giảm xuống 256 cho nhẹ.
    print("Sinh tham số DSA (demo, chậm một chút)...")
    p, q, g = gen_dsa_params(q_bits=160, p_bits=512)

    print("p =", p)
    print("q =", q)
    print("g =", g)

    print("\nSinh cặp khóa...")
    x, y = dsa_keygen(p, q, g)
    print("Khóa bí mật x =", x)
    print("Khóa công khai y =", y)

    message = b"tran duc linh"
    print("\nThông điệp:", message)

    print("Ký...")
    r, s = dsa_sign(message, p, q, g, x)
    print("Chữ ký (r, s) =", (r, s))

    print("Verify...")
    ok = dsa_verify(message, (r, s), p, q, g, y)
    print("Kết quả verify:", ok)


Sinh tham số DSA (demo, chậm một chút)...
p = 7920894203199838861265982292150145073639791884964996801512487973070337136307597813705796918232360348736195935729860815313093762917710267589799279962425521
q = 1160989911596268367751034078172723496726916617567
g = 4652983669396737584701841422733634742870853936432182037996848622045443364166594449682018916183100829237975680302076349348086458858022805493785235156981442

Sinh cặp khóa...
Khóa bí mật x = 137625536005669386662819622605583539670506342351
Khóa công khai y = 6199606550569527809296329498707261555969789248386914989550260032515123953186534350876809905783115041714104955979953237074585760160690754716716368656321960

Thông điệp: b'tran duc linh'
Ký...
Chữ ký (r, s) = (348907314612214121345204344675322342384381370984, 52105756772725023174186534590888635791195859204)
Verify...
Kết quả verify: True
